# Common imports & library functions

In [11]:
import collections
from collections import defaultdict, Counter
from dataclasses import dataclass
import doctest
import functools
import itertools
from itertools import count
import math
import re
from copy import deepcopy
from utils import product
from enum import Enum

# Day 16: Ticket Translation

In [44]:
def parse_ticket(line):
    """
    >>> parse_ticket('1,5,99')
    [1, 5, 99]
    """
    return [int(i) for i in line.strip().split(',')]

def parse_ranges(line):
    """
    >>> name, values = parse_ranges('class: 50-55 or 587-590')
    >>> name
    'class'
    >>> list(sorted(values))
    [50, 51, 52, 53, 54, 55, 587, 588, 589, 590]
    """
    name, ranges = line.split(': ')
    ranges = ranges.split(' or ')
    values = set()
    for lo_hi in ranges:
        lo, hi = lo_hi.split('-')
        values.update(range(int(lo), int(hi) + 1))
    return name, values

def parse_input(input):
    valid_ranges = {}
    your_ticket = None
    other_tickets = []
    for line in input:
        line = line.strip()
        if ': ' in line:
            name, values = parse_ranges(line)
            valid_ranges[name] = values
        elif ',' in line:
            ticket = parse_ticket(line)
            if your_ticket is None:
                your_ticket = ticket
            else:
                other_tickets.append(ticket)
    return valid_ranges, your_ticket, other_tickets

def invalid_ticket_values(valid_ranges, tickets):
    valid_values = functools.reduce(set.union, valid_ranges.values(), set())
    for ticket_value in itertools.chain(*tickets):
        if ticket_value not in valid_values:
            yield ticket_value

In [45]:
doctest.run_docstring_examples(parse_ticket, globs=None, verbose=True)
doctest.run_docstring_examples(parse_ranges, globs=None, verbose=True)

Finding tests in NoName
Trying:
    parse_ticket('1,5,99')
Expecting:
    [1, 5, 99]
ok
Finding tests in NoName
Trying:
    name, values = parse_ranges('class: 50-55 or 587-590')
Expecting nothing
ok
Trying:
    name
Expecting:
    'class'
ok
Trying:
    list(sorted(values))
Expecting:
    [50, 51, 52, 53, 54, 55, 587, 588, 589, 590]
ok


In [46]:
notes = """
    class: 1-3 or 5-7
    row: 6-11 or 33-44
    seat: 13-40 or 45-50

    your ticket:
    7,1,14

    nearby tickets:
    7,3,47
    40,4,50
    55,2,20
    38,6,12
""".splitlines()

valid_ranges, your_ticket, other_tickets = parse_input(notes)
assert sum(invalid_ticket_values(valid_ranges, other_tickets)) == 71

In [85]:
def valid_tickets(valid_ranges, tickets):
    valid_values = functools.reduce(set.union, valid_ranges.values(), set())
    for ticket in tickets:
        if all(v in valid_values for v in ticket):
            yield ticket

def guess_field(valid_ranges, values):
    for name, valid in valid_ranges.items():
        if all(v in valid for v in values):
            yield name

def first(s):
    return list(s)[0]

def guess_fields(valid_ranges, your_ticket, other_tickets):
    # Step 1: Come up with possible guesses based on which fields' valid ranges are
    # compatible with the values encountered in tickets.
    guesses = []
    tickets = valid_tickets(valid_ranges, other_tickets + [your_ticket])
    for values in zip(*tickets):
        guesses.append(set(guess_field(valid_ranges, values)))

    # Step 2: Prune guesses until there's only one choice for each field, or a contradiction 
    # is hit (an empty set is left).
    while True:
        updated = False
        
        for field_i, choices_i in enumerate(guesses):
            if len(choices_i) == 0:
                raise Exception(f'No guesses remaining for field {field_i}')
            elif len(choices_i) == 1:
                guess = first(choices_i)
                for field_j, choices_j in enumerate(guesses):
                    if field_i == field_j:
                        continue
                    elif guess in choices_j:
                        choices_j.remove(guess)
                        updated = True
            
        if not updated:
            return [first(choices) for choices in guesses]


In [89]:
notes = """
    class: 0-1 or 4-19
    row: 0-5 or 8-19
    seat: 0-13 or 16-19

    your ticket:
    11,12,13

    nearby tickets:
    3,9,18
    15,1,5
    5,14,9
""".splitlines()

valid_ranges, your_ticket, other_tickets = parse_input(notes)
assert guess_fields(valid_ranges, your_ticket, other_tickets) == ['row', 'class', 'seat']

In [93]:
%%time
# Final answers
with open('day16.txt') as f:
    notes = f.readlines()
    valid_ranges, your_ticket, other_tickets = parse_input(notes)
    print('Part 1: ', sum(invalid_ticket_values(valid_ranges, other_tickets)))
    
    fields = guess_fields(valid_ranges, your_ticket, other_tickets)
    print('Part 2: ', product(value for field, value in zip(fields, your_ticket) if field.startswith('departure')))

Part 1:  32842
Part 2:  2628667251989
Wall time: 13 ms


# Day 17: Conway Cubes

In [159]:
ds = (-1, 0, 1)

class CubeState(Enum):
    INACTIVE = 0
    ACTIVE = 1

CubeGrid = lambda init={}: defaultdict(lambda: CubeState.INACTIVE, init)

def neighboring_cells(cell):
    """
    >>> list(neighboring_cells((0, 0)))
    [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    >>> len(list(neighboring_cells((0, 0, 0))))
    26
    >>> len(list(neighboring_cells((-1, 0, 0, 1))))
    80
    """
    for deltas in itertools.product(*[ds]*len(cell)):
        if all(d == 0 for d in deltas): continue
        yield tuple(c+d for c, d in zip(cell, deltas))

def parse_state(state_2d, dims=3):
    rows = state_2d.strip().splitlines()
    state_3d = CubeGrid()
    for y, row in enumerate(rows):
        for x, state in enumerate(row.strip()):
            coord = (x, y) + (0,) * (dims - 2)
            state_3d[coord] = CubeState.INACTIVE if state == '.' else CubeState.ACTIVE
    return state_3d

def num_active(state):
    return sum(cell_state.value for cell_state in state.values())

def step(initial_state, n_steps=1):
    """
    >>> initial_state = parse_state('''
    ...    .#.
    ...    ..#
    ...    ###
    ... ''')
    >>> num_active(step(initial_state, n_steps=6))
    112
    """
    prev_state = CubeGrid(initial_state)
    
    while n_steps > 0:
        # Visit cells and their neighbors this iteration, extending the boundary
        # of visited cells by one.
        to_visit = set(prev_state.keys())
        for cell in prev_state:
            to_visit.update(neighboring_cells(cell))
        #print('Steps remaining', n_steps, ', cells to visit', len(to_visit))

        # Perform a single update step.
        next_state = CubeGrid()
        for cell in to_visit:
            state = prev_state[cell]
            # Count the number of living neighbors and update current state.
            live_neighbors = sum(prev_state[n].value for n in neighboring_cells(cell))
            if state == CubeState.ACTIVE and live_neighbors not in (2, 3):
                next_state[cell] = CubeState.INACTIVE
            elif state == CubeState.INACTIVE and live_neighbors == 3:
                next_state[cell] = CubeState.ACTIVE
            else:
                next_state[cell] = state

        prev_state = next_state
        n_steps -= 1
    
    return prev_state



In [160]:
doctest.run_docstring_examples(neighboring_cells, globs=None, verbose=True)
doctest.run_docstring_examples(step, globs=None, verbose=True)

Finding tests in NoName
Trying:
    list(neighboring_cells((0, 0)))
Expecting:
    [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
ok
Trying:
    len(list(neighboring_cells((0, 0, 0))))
Expecting:
    26
ok
Trying:
    len(list(neighboring_cells((-1, 0, 0, 1))))
Expecting:
    80
ok
Finding tests in NoName
Trying:
    initial_state = parse_state('''
       .#.
       ..#
       ###
    ''')
Expecting nothing
ok
Trying:
    num_active(step(initial_state, n_steps=6))
Expecting:
    112
ok


In [161]:
%%time
# Final answers
with open('day17.txt') as f:
    initial_state_2d = f.read()
    initial_state_3d = parse_state(initial_state_2d, dims=3)
    print('Part 1: ', num_active(step(initial_state_3d, 6)))
    
    initial_state_4d = parse_state(initial_state_2d, dims=4)
    print('Part 2: ', num_active(step(initial_state_4d, 6)))

Part 1:  306
Part 2:  2572
Wall time: 47.8 s
